In [ ]:
import pandas as pd   #preprocessing
import numpy as np
import math
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt    
import torch          #modelling
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset
from collections import defaultdict


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from IMV_LSTM import IMVTensorLSTM
from IMV_LSTM import IMVFullLSTM
from boruta import BorutaPy
from BorutaShap import BorutaShap
import shap
import shap.plots


#import operator
import time
import os
import operator

from IPython.utils import io
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 


In [ ]:
N_EPOCHS = 1001
BATCH_SIZE = 7
SEQUENCE_LENGTH = 15
N_HIDDEN = 32
N_LAYERS = 1
PATIENCE = 50
LEARNING = 0.001

folder_path1='02-Results/FS'
TGT = 'Sales'

Correlation = False

BorutaGB = False
BorutaRF = False

BorutaSHAPGB = False
BorutaSHAPRF = False

LSTM_SHAP = False
LSTM_LIME_ = True

IMV_Tensor = False
IMV_Full = False

In [ ]:
class Dataset(Dataset):
    def __init__(self,sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self,idx):
        sequence, label = self.sequences[idx]

        return dict(sequence = torch.Tensor(sequence.to_numpy()),
            label = torch.tensor(label).float())

class DataModule(pl.LightningDataModule):
    def __init__(self, train_seqeunces,test_sequences, batch_size=8):
        super().__init__()
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size

    def setup(self,stage=None):
        self.train_dataset = Dataset(self.train_sequences)
        self.test_dataset = Dataset(self.test_sequences)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size = self.batch_size,
            shuffle = False,
            num_workers = 0)

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = 1,
            shuffle = False,
            num_workers = 0)
    
    
class PredictionModel(nn.Module):
    def __init__(self, n_features, n_hidden = N_HIDDEN, n_layers = N_LAYERS):
        super().__init__()
        self.n_hidden = n_hidden
        self.lstm = nn.LSTM(
            input_size = n_features,
            hidden_size = n_hidden,
            batch_first = True,
            num_layers = n_layers,
            dropout = 0.2)
        self.regressor = nn.Linear(n_hidden,1)

    def forward(self,x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        out = hidden[-1]
        return self.regressor(out)


class Predictor(pl.LightningModule):
    def __init__(self, n_features: int):
        super().__init__()
        self.model=PredictionModel(n_features)
        self.criterion = nn.MSELoss()

    def forward(self, x, labels= None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.unsqueeze(dim=1))
        return loss, output

    def training_step(self, batch, batch_index):
        sequences = batch['sequence']
        labels = batch['label']
        loss, outputs = self(sequences, labels)
        self.log('train_loss', loss, prog_bar = True, logger=False)
        return loss

    def validation_step(self, batch, batch_index):
        sequences = batch['sequence']
        labels = batch['label']
        loss, outputs = self(sequences, labels)
        self.log('val_loss', loss, prog_bar = True, logger=False)
        return loss

    def test_step(self, batch, batch_index):
        sequences = batch['sequence']
        labels = batch['label']
        loss, outputs = self(sequences, labels)
        self.log('test_loss', loss, prog_bar = True, logger=False)
        return loss

    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr = LEARNING)


In [ ]:
def features_dataframe(df,features):
    rows = []

    for _, row in df.iterrows():
        row_data = dict(
            Sales = df[f'{TGT}'],
        )
        for column in features:
            row_data[column] = row[column]
            
        rows.append(row_data)
    
    features_df = pd.DataFrame(rows)

    return features_df

#spliits the data in test and train
def train_test_spliter(ratio,features_df ):
    train_size = int(len(features_df)-ratio)
    train_df, test_df = features_df[:train_size], features_df[train_size + 1:]

    return train_df, test_df, train_size

def data_scaler(train_df,test_df):
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler = scaler.fit(train_df)
    
    train_df = pd.DataFrame(
        scaler.transform(train_df),
        index = train_df.index,
        columns = train_df.columns)

    test_df = pd.DataFrame(
        scaler.transform(test_df),
        index = test_df.index,
        columns = test_df.columns)
    
    return train_df, test_df, scaler


def create_sequences (input_data:pd.DataFrame, target_column, sequence_length):
    sequences = []
    data_size = len(input_data)

    for i in (range(data_size - sequence_length)):
        sequence = input_data[i:i+sequence_length]
        label_position = i + sequence_length
        label = input_data.iloc[label_position][target_column]
        sequences.append((sequence,label))
    
    return sequences


In [ ]:
def pep_proc_summarizer(features,df):
    features_df = features_dataframe(df,features) 
    #returns dataframe with the features to be analised
    
    #split into test and train and minmaxscaler
    train_df, test_df, train_size =  train_test_spliter(105,features_df)
    train_df, test_df, scaler = data_scaler(train_df,test_df)
    
    #make sequences with the data
    train_sequences = create_sequences(train_df,TGT,SEQUENCE_LENGTH)
    test_sequences = create_sequences (test_df,TGT,SEQUENCE_LENGTH)
    train_dataset = Dataset(train_sequences)
    test_dataset = Dataset(test_sequences)
    
    return (train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler)

def dict_to_dataframe(dict_list,seq_size):
    column_names = []
    data = []
    for d in dict_list:
        features = d['sequence'].numpy().flatten()
        label = d['label'].numpy()
        data.append(np.concatenate((features, label), axis=None))
        
    df = pd.DataFrame(data)
    for j in range(seq_size):
        if seq_size-j >= 10:
            day_column_names = str(seq_size-j) + 'day_back'+'_' + train_df.columns
            column_names.extend(day_column_names)
        else:
            day_column_names = '0' + str(seq_size-j) + 'day_back'+'_' + train_df.columns
            column_names.extend(day_column_names)
        
    cols = [column_names + ['target']]
    df.columns = cols
    return df

def dict_to_dataframeBoSHAP(dict_list,seq_size):
    column_names = []
    data = []
    for d in dict_list:
        features = d['sequence'].numpy().flatten()
        label = d['label'].numpy()
        data.append(np.concatenate((features, label), axis=None))
        
    df = pd.DataFrame(data)
    for j in range(seq_size):
        if seq_size-j >= 10:
            day_column_names = str(seq_size-j) + 'day_back'+'_' + train_df.columns
            column_names.extend(day_column_names)
        else:
            day_column_names = '0' + str(seq_size-j) + 'day_back'+'_' + train_df.columns
            column_names.extend(day_column_names)
        
    column_names.append('target')
    df.columns = column_names
    return df

def LSTM_predict(array):
    with torch.no_grad():

        predictions_lime= list(array[:SEQUENCE_LENGTH,0])
        label_lime=[]

        train_df = pd.DataFrame(array)
        #features_df.columns = features

        train_sequences = create_sequences(train_df,0,SEQUENCE_LENGTH)

        data_module = DataModule(train_sequences, test_sequences, batch_size = BATCH_SIZE)

        train_dataset = Dataset(train_sequences)

        data_module.setup()

        for item in train_dataset:
            sequence = item['sequence']
            label = item['label']
            _,output = trained_model(sequence.unsqueeze(dim=0))
            predictions_lime.append(output.item())
            label_lime.append(label.item())

        output = np.array(predictions_lime)

    return output


def LIME_Explainer(dates):
    features_evo = {col: [] for col in train_df.columns}
    
    for local in tqdm(range(dates,0,-1)):
        lst = explainer.explainer.explain_instance(train_df.iloc[-local],LSTM_predict).as_list()

        result = [(tup[0].split()) for tup in lst]

        for i in range(len(result)):
            if len(result[i]) == 3:
                features_evo[(train_df.columns[int(result[i][0])])].append(lst[i][1])
            else:
                features_evo[(train_df.columns[int(result[i][2])])].append(lst[i][1])

        for key in features_evo.keys():
            if len(features_evo[key]) < (dates - local)+1:
                features_evo[key].append(0)
                
    return features_evo


def return_shap_features(shap_values,train_df, number_to_return):
    shapdict = {}
    for k in train_df.columns:
        shapdict[k] = 0
        shapdict[f'{k}_count'] = 0


    for i in shap_values:
        for f in range(len(train_df.columns)):
            shapdict[train_df.columns[f]] += abs(i[f])
            shapdict[f'{train_df.columns[f]}_count'] += 1

    avg_n0_shap = {}

    for key in range(0,len(shapdict.keys()),2):
        if shapdict[list(shapdict.keys())[key+1]] != 0 and shapdict[list(shapdict.keys())[key]] != 0:
            avg_n0_shap[list(shapdict.keys())[key]] = shapdict[list(shapdict.keys())[key]]/shapdict[list(shapdict.keys())[key+1]]

    sorted_avg_shap = dict(sorted(avg_n0_shap.items(), key=operator.itemgetter(1), reverse=True))
    
    top_features = dict(sorted(avg_n0_shap.items(), key=lambda item: item[1], reverse=True)[:number_to_return])
    
    return sorted_avg_shap, top_features


    
def return_shap_max(shap_values,train_df):
    shapdict = {}
    for k in train_df.columns:
        shapdict[k] = []

    for i in shap_values:
        for f in range(len(train_df.columns)):
            shapdict[train_df.columns[f]].append(abs(i[f]))

    max_shap = {}
    for key in shapdict.keys():
        max_shap[key] = max(shapdict[key])

    max_shap_df = pd.DataFrame.from_dict(max_shap, orient='index', columns=['max_shap_value'])
    max_shap_df.index.name = 'feature_name'
    max_shap_df = max_shap_df.sort_values(by='max_shap_value', ascending=False)

    return max_shap_df

In [ ]:
Rossmann = pd.read_csv('01-Data/Rossmann_treated.csv')
Rosmann_stores = [23,64,103,133,135,216,224,256,266,311,355,405,455,487,558,649,672,714,738,742,773,785,
                 830,870,880,899,937,978,1011,1069]
Rossmann.drop(columns = 'Customers', inplace = True)
Rossmann.drop(columns = 'Date', inplace = True)

try:
    os.mkdir(f'{folder_path1}')
except:
    pass

for store in tqdm(Rosmann_stores):
    try:
        os.mkdir(f'{folder_path1}/Store{store}')
    except:
        pass

## PCC Feature Selection

In [ ]:
if Correlation == True:
    corr_values = [0.05,0.10,0.15,0.20,0.25]
    run_time = []
    durations={}
    
    for store in tqdm(Rosmann_stores):            
        folder_path2=f'Store{store}/Correlation'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
    
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        correlation_df = pd.DataFrame(list(Rossmann_df.columns),columns = ['all_features'])
        
        
        iteration_start = time.monotonic()
        for cor_th in corr_values:  
            Tgt_corr = Rossmann_df.corr()[abs(Rossmann_df.corr()) > cor_th]
            features = Tgt_corr[Tgt_corr[TGT].notnull()].index
            df = pd.DataFrame(features, columns = [f'correlation_th_=_{cor_th}'])
            correlation_df = correlation_df.join(df,how='left')
        iteration_end = time.monotonic()
        correlation_df.to_csv(f"{folder_path1}/{folder_path2}/correlation_df.csv",index = False)
        
        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/Correlation_time.csv")

## Boruta-GB Feature Selection

In [ ]:
if BorutaGB == True:    
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42);
        
        folder_path2=f'Store{store}/Boruta-GB'
        
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        estimator = GradientBoostingRegressor(n_estimators=100,
                                              max_depth=3,
                                              learning_rate=0.1,
                                              loss='squared_error',
                                              criterion='friedman_mse',
                                              min_samples_split=2,
                                              min_samples_leaf=1)
        
        boruta = BorutaPy(estimator = estimator,n_estimators = 30, max_iter =100)
        
        
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
        data = dict_to_dataframe(train_dataset, SEQUENCE_LENGTH)


        iteration_start = time.monotonic()
        boruta.fit(np.array(data.iloc[:,:-1]),np.array(data.iloc[:,-1]))
        iteration_end = time.monotonic()

        important = list(data.iloc[:,:-1].columns[boruta.support_])
        important = [i[0][11:] for i in important]
        important = list(set(important)) 

        unimportant = list(data.iloc[:,:-1].columns[boruta.support_weak_])
        unimportant = [i[0][11:] for i in unimportant]
        unimportant = list(set(unimportant))

        tentative = list(data.iloc[:,:-1].columns[~(boruta.support_|boruta.support_weak_)])
        tentative = [i[0][11:] for i in tentative]
        tentative = list(set(tentative)) 

        for i in important:
            if i in unimportant:
                unimportant.remove(i)
        for i in important:
            if i in tentative:
                tentative.remove(i)
        for i in unimportant:
            if i in tentative:
                tentative.remove(i)

        important_df = pd.DataFrame(important, columns=[f'important'])
        unimportant_df = pd.DataFrame(unimportant, columns=[f'unimportant'])
        tentative_df = pd.DataFrame(tentative, columns=[f'tentative'])
        borutaGB_df = pd.DataFrame(list(Rossmann_df.columns),columns = ['all_features'])
        borutaGB_df = borutaGB_df.join([important_df,unimportant_df, tentative_df],how='left')
        borutaGB_df.to_csv(f"{folder_path1}/{folder_path2}/borutaGB_df.csv",index = False)

        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/Boruta-GB_time.csv")

## Boruta-RF Feature Selection

In [ ]:
if BorutaRF == True:  
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42);
        
        folder_path2=f'Store{store}/Boruta-RF'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        estimator = RandomForestRegressor(n_estimators=100,
                                          min_samples_split=2,
                                          min_samples_leaf=1,
                                          max_depth=3,
                                          criterion='squared_error')
        boruta = BorutaPy(estimator = estimator,n_estimators = 30, max_iter = 100)
        
        
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
        data = dict_to_dataframe(train_dataset, SEQUENCE_LENGTH)


        iteration_start = time.monotonic()
        boruta.fit(np.array(data.iloc[:,:-1]),np.array(data.iloc[:,-1]))
        iteration_end = time.monotonic()

        important = list(data.iloc[:,:-1].columns[boruta.support_])
        important = [i[0][11:] for i in important]
        important = list(set(important)) 

        unimportant = list(data.iloc[:,:-1].columns[boruta.support_weak_])
        unimportant = [i[0][11:] for i in unimportant]
        unimportant = list(set(unimportant))

        tentative = list(data.iloc[:,:-1].columns[~(boruta.support_|boruta.support_weak_)])
        tentative = [i[0][11:] for i in tentative]
        tentative = list(set(tentative)) 

        for i in important:
            if i in unimportant:
                unimportant.remove(i)
        for i in important:
            if i in tentative:
                tentative.remove(i)
        for i in unimportant:
            if i in tentative:
                tentative.remove(i)

        important_df = pd.DataFrame(important, columns=[f'important'])
        unimportant_df = pd.DataFrame(unimportant, columns=[f'unimportant'])
        tentative_df = pd.DataFrame(tentative, columns=[f'tentative'])
        
        BorutaRF_df = pd.DataFrame(list(Rossmann_df.columns),columns = ['all_features'])
        BorutaRF_df = BorutaRF_df.join([important_df,unimportant_df, tentative_df],how='left')
        BorutaRF_df.to_csv(f"{folder_path1}/{folder_path2}/BorutaRF_df.csv",index = False)

        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/Boruta-RF_time.csv")

## BorutaSHAP-GB Feature Selection

In [ ]:
if BorutaSHAPGB == True:
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42);
        
        folder_path2=f'Store{store}/BorutaSHAP-GB'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        estimator = GradientBoostingRegressor(n_estimators=100,
                                              max_depth=3,
                                              learning_rate=0.1,
                                              loss='squared_error',
                                              criterion='friedman_mse',
                                              min_samples_split=2,
                                              min_samples_leaf=1)
    
        selector = BorutaShap(estimator,importance_measure = 'shap', classification = False)     
    
    
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
        data = dict_to_dataframeBoSHAP(train_dataset, SEQUENCE_LENGTH)

        iteration_start = time.monotonic()
        with io.capture_output() as captured:
            selector.fit(data.iloc[:,:-1], data.iloc[:,-1], n_trials=20,
                        sample=False, train_or_test='train');
        iteration_end = time.monotonic()
        
        important = list(selector.accepted)
        important = [i[11:] for i in important]
        important = list(set(important)) 

        unimportant = list(selector.rejected)
        unimportant = [i[11:] for i in unimportant]
        unimportant = list(set(unimportant))
        
        tentative = list(selector.tentative)
        tentative = [i[11:] for i in tentative]
        tentative = list(set(tentative))

        
        for i in important:
            if i in unimportant:
                unimportant.remove(i)
        for i in important:
            if i in tentative:
                tentative.remove(i)
        for i in unimportant:
            if i in tentative:
                tentative.remove(i)

        important_df = pd.DataFrame(important, columns=[f'important'])
        unimportant_df = pd.DataFrame(unimportant, columns=[f'unimportant'])
        tentative_df = pd.DataFrame(tentative, columns=[f'tentative'])
        
        borutaSHAPGB_df = pd.DataFrame(list(Rossmann_df.columns),columns = ['all_features'])
        borutaSHAPGB_df = borutaSHAPGB_df.join([important_df,unimportant_df, tentative_df],how='left')
        borutaSHAPGB_df.to_csv(f"{folder_path1}/{folder_path2}/borutaSHAPGB_df.csv",index = False)

        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/BorutaSHAP-GB_time.csv")

## BorutaSHAP-RF Feature Selection

In [ ]:
if BorutaSHAPRF == True:  
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42);

        folder_path2=f'Store{store}/BorutaSHAP-RF'
        
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        estimator = RandomForestRegressor(n_estimators=100,
                                          min_samples_split=2,
                                          min_samples_leaf=1,
                                          max_depth=3,
                                          criterion='squared_error')
    
        selector = BorutaShap(estimator,importance_measure = 'shap', classification = False)     
    
    
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
        data = dict_to_dataframeBoSHAP(train_dataset, SEQUENCE_LENGTH)

        iteration_start = time.monotonic()
        with io.capture_output() as captured:
            selector.fit(data.iloc[:,:-1], data.iloc[:,-1], n_trials=20,
                        sample=False, train_or_test='train');
        iteration_end = time.monotonic()
        
        important = list(selector.accepted)
        important = [i[11:] for i in important]
        important = list(set(important)) 

        unimportant = list(selector.rejected)
        unimportant = [i[11:] for i in unimportant]
        unimportant = list(set(unimportant))
        
        tentative = list(selector.tentative)
        tentative = [i[11:] for i in tentative]
        tentative = list(set(tentative))

        
        for i in important:
            if i in unimportant:
                unimportant.remove(i)
        for i in important:
            if i in tentative:
                tentative.remove(i)
        for i in unimportant:
            if i in tentative:
                tentative.remove(i)

        important_df = pd.DataFrame(important, columns=[f'important'])
        unimportant_df = pd.DataFrame(unimportant, columns=[f'unimportant'])
        tentative_df = pd.DataFrame(tentative, columns=[f'tentative'])
        
        borutaSHAPRF_df = pd.DataFrame(list(Rossmann_df.columns),columns = ['all_features'])
        borutaSHAPRF_df = borutaSHAPRF_df.join([important_df,unimportant_df, tentative_df],how='left')
        borutaSHAPRF_df.to_csv(f"{folder_path1}/{folder_path2}/borutaSHAPRF_df.csv",index = False)

        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/BorutaSHAP-RF_time.csv")



## SHAP-LSTM FS

In [ ]:
if LSTM_SHAP == True: 
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42);  
        
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
        
        folder_path2=f'Store{store}/SHAP-LSTM'
        
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        kmeans = shap.kmeans(train_df,5)
        #explainer
        
        trained_model = Predictor.load_from_checkpoint(
        f'02-Results/LSTM/Store{store}/All_Features/Checkpoints/Rossmann_All_LSTM_epoch=199.ckpt',
        n_features = train_df.shape[1]
        )

        iteration_start = time.monotonic()
        explainer = shap.KernelExplainer(LSTM_predict, kmeans)
        #explanation
        shap_values = explainer.shap_values(train_df[-365:])
        iteration_end = time.monotonic()
        
        
        #return a FEATURES_TO_USE numpber of features ordered by average shap value
        sorted_avg_shap, top_features_dic = return_shap_features(shap_values,train_df,30)
        shap_per_feature = pd.DataFrame.from_dict(sorted_avg_shap, orient='index', columns=['Importance'])
        shap_per_feature.to_csv(f"{folder_path1}/{folder_path2}/avg_SHAP_LSTM.csv")
        max_shap_df = return_shap_max (shap_values,train_df)
        max_shap_df.to_csv(f"{folder_path1}/{folder_path2}/inst_SHAP_LSTM.csv")
        ExplanationSHAP = shap.Explanation(values=shap_values, data=train_df[-365:])


        run_time.append(iteration_end - iteration_start)
        
    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/SHAP_LSTM_time.csv")



## LIME-LSTM FS

In [ ]:
if LSTM_LIME_ == True:
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42); 
        
        Rossmann_df = Rossmann[Rossmann['Store'] == store]
        features  = list(Rossmann_df.columns)
        train_df,train_sequences,test_sequences,train_size,train_dataset,test_dataset,scaler=pep_proc_summarizer(features,Rossmann_df)
                
        dic={}
        trained_model = Predictor.load_from_checkpoint(
        f'02-Results/LSTM/Store{store}/All_Features/Checkpoints/Rossmann_All_LSTM_epoch=199.ckpt',
        n_features = train_df.shape[1]
        )
        
        
        folder_path2=f'Store{store}/LIME-LSTM'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass
        
        iteration_start = time.monotonic()
        explainer = shap.explainers.other.LimeTabular(LSTM_predict,train_df[-365:],'regression')
        lime = LIME_Explainer(60)
        iteration_end = time.monotonic()
        
        lime_df = pd.DataFrame(lime)
        lime_df_single = np.abs(lime_df)

        for col in lime_df.columns:
            lime_df_single[col] = np.sum(np.abs(lime_df_single[col]))/len(lime_df_single[col])
            
        lime_df_single = lime_df_single.iloc[0]
        lime_df_single  = lime_df_single.sort_values(ascending=False)

        lime_df_single.to_csv(f"{folder_path1}/{folder_path2}/LIME-LSTM.csv", header = ['LIME_value'],index_label="Features")
        
        for col in lime_df.columns:
            dic[col]=[]
        for i in range(2,len(lime_df)):
            for col in lime_df.columns:
                dic[col].append(np.sum(np.abs(lime_df[col].iloc[:i]))/len(lime_df[col].iloc[:i]))
                
        large_n_rule =  pd.DataFrame.from_dict(dic)
        large_n_rule.to_csv(f"{folder_path1}/{folder_path2}/large_n_rule.csv",index = False)
        run_time.append(iteration_end - iteration_start)

    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/LIME_LSTM_time.csv")
    



## IMV-LSTM 

In [ ]:
def preprop_imv(store):
    df_og = Rossmann[Rossmann['Store'] == store]

    data1=df_og.iloc[:-89,:]
    data2=df_og.iloc[-89:,:]

    target = TGT
    cols = list(data1.columns)

    train_size = data1.shape[0]
    val_size = 89
    depth = 15
    batch_size = 7
    prediction_horizon = 1

    X_train1 = np.zeros((len(data1), depth, len(cols)))
    y_train1 = np.zeros((len(data1), 1))

    for i, name in enumerate(cols):
        for j in range(depth):
            X_train1[:, j, i] = data1[name].shift(depth - j - 1).fillna(method="bfill")
    y_train1 = data1[target].shift(-prediction_horizon).fillna(method='ffill')

    X_train1 = X_train1[depth:-prediction_horizon]
    y_train1 = y_train1[depth:-prediction_horizon]

    X2 = np.zeros((len(data2), depth, len(cols)))
    y2 = np.zeros((len(data2), 1))

    for i, name in enumerate(cols):
        for j in range(depth):
            X2[:, j, i] = data2[name].shift(depth - j - 1).fillna(method="bfill")
    y2 = data2[target].shift(-prediction_horizon).fillna(method='ffill')

    X_train2 = X2[:train_size - len(data1)]
    y_train2 = y2[:train_size - len(data1)]

    X_val = X2[train_size - len(data1):train_size - len(data1) + val_size]
    y_val = y2[train_size - len(data1):train_size - len(data1) + val_size]

    X_test = X2[train_size - len(data1) + val_size:]
    y_test = y2[train_size - len(data1) + val_size:]

    X_train2 = X_train2[depth:]
    y_train2 = y_train2[depth:]

    X_train = np.concatenate([X_train1, X_train2], axis=0)
    y_train = np.concatenate([y_train1, y_train2], axis=0)

    X_train_min, y_train_min = X_train.min(axis=0), y_train.min(axis=0)
    X_train_max, y_train_max = X_train.max(axis=0), y_train.max(axis=0)

    X_train = (X_train - X_train_min)/(X_train_max - X_train_min + 1e-9)
    X_val = (X_val - X_train_min)/(X_train_max - X_train_min + 1e-9)
    X_test = (X_test - X_train_min)/(X_train_max - X_train_min + 1e-9)

    y_train = (y_train - y_train_min)/(y_train_max - y_train_min + 1e-9)
    y_val = (y_val - y_train_min)/(y_train_max - y_train_min + 1e-9)
    y_test = (y_test - y_train_min)/(y_train_max - y_train_min + 1e-9)

    X_train_t = torch.Tensor(X_train)
    X_val_t = torch.Tensor(X_val)
    X_test_t = torch.Tensor(X_test)
    y_train_t = torch.Tensor(y_train)
    y_val_t = torch.Tensor(y_val.values)
    y_test_t = torch.Tensor(y_test.values)

    train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), shuffle=True, batch_size=batch_size)
    val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), shuffle=False, batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), shuffle=False, batch_size=batch_size)
    
    return(X_train,train_loader,val_loader,test_loader,X_train_t,X_val_t,X_test_t,y_train_t,y_val_t,y_test_t,y_train_max,y_train_min,cols)


In [ ]:
epochs = 100
loss = nn.MSELoss()
patience = 20


if IMV_Tensor == True:
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42); 
        min_val_loss = 9999
        counter = 0
        X_train,train_loader,val_loader,test_loader,X_train_t,X_val_t,X_test_t,y_train_t,y_val_t,y_test_t,y_train_max,y_train_min,cols = preprop_imv(store)


        folder_path2=f'Store{store}/IMV-LSTM'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass

        model = IMVTensorLSTM(X_train.shape[2], 1, 32)
        opt = torch.optim.Adam(model.parameters(), lr=0.05)
        epoch_scheduler = torch.optim.lr_scheduler.StepLR(opt, 20, gamma=0.9)

        iteration_start = time.monotonic()
        for i in range(epochs):
            mse_train = 0
            for batch_x, batch_y in train_loader :
                batch_x = batch_x
                batch_y = batch_y
                opt.zero_grad()
                y_pred, alphas_Tensor, betas_Tensor = model(batch_x)
                y_pred = y_pred.squeeze(1)
                l = loss(y_pred, batch_y)
                l.backward()
                mse_train += l.item()*batch_x.shape[0]
                opt.step()
            epoch_scheduler.step()
            #with torch.no_grad():
            mse_val = 0
            preds = []
            true = []
            alphas_Tensor = []
            betas_Tensor = []

            for batch_x, batch_y in val_loader:
                batch_x = batch_x
                batch_y = batch_y
                output, a, b = model(batch_x)
                output = output.squeeze(1)
                preds.append(output.detach().cpu().numpy())
                true.append(batch_y.detach().cpu().numpy())
                alphas_Tensor.append(a.detach().cpu().numpy())
                betas_Tensor.append(b.detach().cpu().numpy())
                mse_val += loss(output, batch_y).item()*batch_x.shape[0]
            preds = np.concatenate(preds)
            true = np.concatenate(true)
            alphas_Tensor = np.concatenate(alphas_Tensor)
            betas_Tensor = np.concatenate(betas_Tensor)

            if min_val_loss > mse_val**0.5:
                min_val_loss = mse_val**0.5
                #print("Saving...")
                torch.save(model.state_dict(), "imv_full_lstm.pt")
                counter = 0
            else: 
                counter += 1

            if counter == patience:
                preds = preds*(y_train_max - y_train_min) + y_train_min
                true = true*(y_train_max - y_train_min) + y_train_min
                mse = mean_squared_error(true, preds)
                mae = mean_absolute_error(true, preds)
                print("mse: ", mse, "mae: ", mae)
                break
    
            if(i % 10 == 0):
                preds = preds*(y_train_max - y_train_min) + y_train_min
                true = true*(y_train_max - y_train_min) + y_train_min
                mse = mean_squared_error(true, preds)
                mae = mean_absolute_error(true, preds)
                #print("mse: ", mse, "mae: ", mae)

        iteration_end = time.monotonic()

        alphas_Tensor = alphas_Tensor.mean(axis=0)
        betas_Tensor = betas_Tensor.mean(axis=0)

        alphas_Tensor = alphas_Tensor[..., 0]
        betas_Tensor = betas_Tensor[..., 0]

        alphas_Tensor = alphas_Tensor.transpose(1, 0)

        X_train_t = torch.Tensor(X_train)

        IMV_T_dic = {'features': cols , 'Importance': betas_Tensor} 
        IMV_T_df = pd.DataFrame(IMV_T_dic)
        IMV_T_df  = IMV_T_df.sort_values(ascending=False, by ='Importance')

        IMV_T_df.to_csv(f"{folder_path1}/{folder_path2}/IMV_Tensor.csv")

        run_time.append(iteration_end - iteration_start)

    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/IMV_Tensor_time.csv")



In [ ]:
epochs = 100
loss = nn.MSELoss()
patience = 20


if IMV_Full == True:
    durations={}
    run_time = []
    for store in tqdm(Rosmann_stores):
        torch.manual_seed(42);
        np.random.seed(42);
        pl.seed_everything(42); 
        min_val_loss = 9999
        counter = 0
        X_train,train_loader,val_loader,test_loader,X_train_t,X_val_t,X_test_t,y_train_t,y_val_t,y_test_t,y_train_max,y_train_min,cols = preprop_imv(store)


        folder_path2=f'Store{store}/IMV-LSTM'
        try:
            os.mkdir(f'{folder_path1}/{folder_path2}')
        except:
            pass

        model = IMVFullLSTM(X_train.shape[2], 1, 32)
        opt = torch.optim.Adam(model.parameters(), lr=0.05)
        epoch_scheduler = torch.optim.lr_scheduler.StepLR(opt, 20, gamma=0.9)

        iteration_start = time.monotonic()
        for i in (range(epochs)):
            mse_train = 0
            for batch_x, batch_y in train_loader :
                batch_x = batch_x
                batch_y = batch_y
                opt.zero_grad()
                y_pred, alphas_Tensor, betas_Tensor = model(batch_x)
                y_pred = y_pred.squeeze(1)
                l = loss(y_pred, batch_y)
                l.backward()
                mse_train += l.item()*batch_x.shape[0]
                opt.step()
            epoch_scheduler.step()
            #with torch.no_grad():
            mse_val = 0
            preds = []
            true = []
            alphas_Tensor = []
            betas_Tensor = []

            for batch_x, batch_y in val_loader:
                batch_x = batch_x
                batch_y = batch_y
                output, a, b = model(batch_x)
                output = output.squeeze(1)
                preds.append(output.detach().cpu().numpy())
                true.append(batch_y.detach().cpu().numpy())
                alphas_Tensor.append(a.detach().cpu().numpy())
                betas_Tensor.append(b.detach().cpu().numpy())
                mse_val += loss(output, batch_y).item()*batch_x.shape[0]
            preds = np.concatenate(preds)
            true = np.concatenate(true)
            alphas_Tensor = np.concatenate(alphas_Tensor)
            betas_Tensor = np.concatenate(betas_Tensor)

            if min_val_loss > mse_val**0.5:
                min_val_loss = mse_val**0.5
                #print("Saving...")
                torch.save(model.state_dict(), "imv_full_lstm.pt")
                counter = 0
            else: 
                counter += 1

            if counter == patience:
                preds = preds*(y_train_max - y_train_min) + y_train_min
                true = true*(y_train_max - y_train_min) + y_train_min
                mse = mean_squared_error(true, preds)
                mae = mean_absolute_error(true, preds)
                print("mse: ", mse, "mae: ", mae)
                break
            #print("Iter: ", i, "train: ", (mse_train/len(X_train_t))**0.5, "val: ", (mse_val/len(X_val_t))**0.5)
            if(i % 10 == 0):
                preds = preds*(y_train_max - y_train_min) + y_train_min
                true = true*(y_train_max - y_train_min) + y_train_min
                mse = mean_squared_error(true, preds)
                mae = mean_absolute_error(true, preds)
                #print("mse: ", mse, "mae: ", mae)

        iteration_end = time.monotonic()

        alphas_Tensor = alphas_Tensor.mean(axis=0)
        betas_Tensor = betas_Tensor.mean(axis=0)

        alphas_Tensor = alphas_Tensor[..., 0]
        betas_Tensor = betas_Tensor[..., 0]

        alphas_Tensor = alphas_Tensor.transpose(1, 0)

        X_train_t = torch.Tensor(X_train)

        IMV_T_dic = {'features': cols , 'Importance': betas_Tensor} 
        IMV_T_df = pd.DataFrame(IMV_T_dic)
        IMV_T_df  = IMV_T_df.sort_values(ascending=False, by ='Importance')

        IMV_T_df.to_csv(f"{folder_path1}/{folder_path2}/IMV_Full.csv")

        run_time.append(iteration_end - iteration_start)

    durations['CompCost'] = run_time
    duration_df =  pd.DataFrame.from_dict(durations)
    duration_df.to_csv(f"02-Results/IMV_Full_time.csv")

